In [1]:
# fast trigonometry

# 1. Arctan

In [2]:
import numpy as np

In [180]:
 
def cubic(x):
    m = x.shape[0]
    X = np.empty((m, 3))
    X[:,0] = x ; X[:,1] = x * x; X[:,2] = x * x * x
    return X

def quad(x):
    m = x.shape[0]
    X = np.empty((m, 2))
    X[:,0] = x ; X[:,1] = x * x;
    return X

def linear(x):
    X = np.expand_dims(x,1)
    return X

In [ ]:
from sklearn.linear_model import LinearRegression
# 1. a + bx + cx^2 + dx^3
_size = 1000000
x = np.clip(np.abs(np.random.randn(_size)) * np.pi, 0, np.pi)
gt = np.arctan(x)

In [32]:
X = cubic(x)
model = LinearRegression()
model.fit(X, gt)
print(model.coef_)
print(model.intercept_)

[ 1.10382257 -0.36105955  0.04400221]
-0.00492334997743038


In [41]:
pred = model.predict(X)
res = (pred - gt)
idx = np.argmax(res)
print("MAX err", "idx", idx, "value", res[idx], "rad", X[idx,0])
predict = cubic(np.array([[2]]))
print(model.predict(predict), " vs ", np.arctan(predict))


MAX err idx 141589 value 0.005227865735183412 rad 1.7133812981746157
[1.11050127]  vs  [[1.10714872 1.32581766 1.44644133]]


In [34]:
lsize = 5000
xl = np.linspace(5,500,lsize)
gtxl = np.arctan(xl)

In [35]:
Xl = cubic(xl) #quad(xl)
modell = LinearRegression()
modell.fit(Xl, gtxl)
print(modell.coef_)
print(modell.intercept_)

[ 7.65325825e-04 -2.76103373e-06  3.04822121e-09]
1.50443794904616


In [36]:
pred = modell.predict(Xl)
res = (pred - gtxl)
idx = np.argmax(res)
print("MAX err", "idx", idx, "value", res[idx], "rad", Xl[idx,0])
predict = cubic(np.array([[5]]))
print(modell.predict(predict), " vs ", np.arctan(predict))

MAX err idx 0 value 0.13479516641067524 rad 5.0
[1.50819593]  vs  [[1.37340077 1.53081764 1.5627965 ]]


In [44]:
val = 6
print(np.arctan(val))
print(np.pi / 2 - 1 / val)

1.4056476493802699
1.4041296601282298


In [85]:
def fast_arctan(x):
    a0 = 1.10382257
    a1 = -0.36105955
    a2 = 0.04400221
    b = -0.00492334997743038
    sign = 1 if x >= 0 else -1
    if (abs(x) > np.pi):
        # https://math.stackexchange.com/questions/982838/asymptotic-approximation-of-the-arctangent/982859
        return (sign*(np.pi / 2) - 1/x)
    elif abs(x) < 5e-2:
        # for small x
        return x
    else:
        # by regression
        ax = abs(x)
        return sign * (b + ax * (a0 + ax * (a1 + a2 * ax)))
        #x2 = ax * ax
        #x3 = x2 * ax
        #return sign * (b + a0 * ax + a1 * x2 + a2 * x3)

In [85]:
val = -0.049
print(np.sin(val), "vs", fast_arctan(val))

-0.04898039418715918 vs -0.049


In [86]:
np.pi

3.141592653589793

# 2. Sine

In [184]:
from sklearn.linear_model import LinearRegression
# 1. a + bx + cx^2 + dx^3
_size = 1000000
x = np.clip(np.abs(np.random.randn(_size)) * np.pi/2, 0, np.pi/2)
gt = np.cos(x)
X = cubic(x)
model = LinearRegression()
model.fit(X, gt)
print(model.coef_)
print(model.intercept_)

[ 0.0385896  -0.61311836  0.11737913]
0.9970072879558844


In [212]:
pred = model.predict(X)
res = np.abs(pred - gt)
idx = np.argmax(res)
print("MAX err", "idx", idx, "value", res[idx], "rad", X[idx,0])
predict = cubic(np.array([[1.5788]]))
print(model.predict(predict), " vs ", np.cos(predict))


MAX err idx 366895 value 0.0029926210732348357 rad 2.3574102242678533e-06
[-0.00840623]  vs  [[-0.00800359 -0.79669873 -0.7011843 ]]


In [186]:
np.pi/2

1.5707963267948966

In [189]:
def fast_cos(x):
    c0 = 0.9970072879558844
    c1 = 0.0385896
    c2 = -0.61311836
    c3 = 0.11737913
    pi_2 = 1.5707963267948966
    ax = abs(x)
    if ax < 0.05:
        return 1
    if ax > pi_2:
        _x = ax / pi_2
        _ix = int(_x)
        _mx = _ix % 4
        ax = (_x - _ix) * pi_2
        sign = 1
        if _mx == 1:
            ax = pi_2 - ax
            sign = -1
        elif _mx == 2:
            sign = -1
        elif _mx == 3:
            ax = pi_2 - ax
    return sign * (c0 + ax * (c1 + ax * (c2 + ax * c3 )))

In [208]:
# https://en.wikipedia.org/wiki/Bhaskara_I%27s_sine_approximation_formula
def fast_sine(x):
    # version one
    pi = 3.141592653589793
    pi2 = pi * pi
    ax = abs(x)
    sign = 1 if x >= 0 else -1
    if ax < 0.05:
        return x
    if ax > pi:
        _x = ax / pi
        _ix = int(_x)
        _mx = _ix % 2
        ax = (_x - _ix) * pi
        sign = sign * (-1 if _mx == 1 else 1)
    return sign * (16 * ax * (pi - ax)) / ( 5 * pi2 - 4 * ax * (pi - ax))


In [209]:
val = -2393923 + 0.23232
print(fast_sine(val), "vs" , np.sin(val))

0.77655256622833 vs 0.7778960471964677


In [210]:
def fast_sineDeg(x):
    ax = abs(x)
    sign = 1 if x >= 0 else -1
    if ax < 2.86:
        return x*3.14159265/180
    elif ax > 180.:
        _x = ax / 180
        _ix = int(_x)
        _mx = _ix % 2
        ax = (_x - _ix) * 180
        sign = sign * (-1 if _mx == 1 else 1)
    return sign * (4 * ax) * (180 - ax) / (40500 - ax * (180 - ax))

In [177]:
val = 3.17
print(fast_sine(val) ,"vs", np.sin(val))

-0.02888087235926067 vs -0.02840352588360379


In [115]:
fast_sineDeg(1)

0.017453292500000002

# 3. Cosine

In [163]:
def fast_cosine(x):
    pi = 3.14159265
    pi_2 = pi / 2
    pi2 = pi*pi
    ax = abs(x)
    sign = 1 if x >= 0 else -1
    if ax < 0.05:
        return 1
    elif ax > pi_2:
        _x = ax / pi_2
        _ix = int(_x)
        _mx = _ix % 4
        ax = (_x - _ix) * pi_2
        sign = 1
        if _mx == 1:
            ax = pi_2 - ax
            sign = -1
        elif _mx == 2:
            sign = -1
        elif _mx == 3:
            ax = pi_2 - ax
        print(_x, _ix, _mx, ax, _x - _ix)
    ax2 = ax * ax
    return sign * (pi2 - 4 * ax2)/(pi2 + ax2)
    

In [213]:
x = 7
print(fast_cosine(x), "vs", np.cos(x), "vs", fast_cos(x), "vs", fast_sine(np.pi/2 - x))

4.456338411665178 4 0 0.7168146999999989 0.4563384116651781
0.7525752914431102 vs 0.7539022543433046 vs 0.7528670014882921 vs 0.7525752966917004


# 4. tangent

# 5. Arcsine

# 6. Arcosine
